# Homework 6

In [1]:
import pandas as pd 

import pandas as pd

url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"

df = pd.read_csv(url)
print(df.head())

   engine_displacement  num_cylinders  horsepower  vehicle_weight  \
0                  170            3.0       159.0     3413.433759   
1                  130            5.0        97.0     3149.664934   
2                  170            NaN        78.0     3079.038997   
3                  220            4.0         NaN     2542.392402   
4                  210            1.0       140.0     3460.870990   

   acceleration  model_year  origin fuel_type         drivetrain  num_doors  \
0          17.7        2003  Europe  Gasoline    All-wheel drive        0.0   
1          17.8        2007     USA  Gasoline  Front-wheel drive        0.0   
2          15.1        2018  Europe  Gasoline  Front-wheel drive        0.0   
3          20.2        2009     USA    Diesel    All-wheel drive        2.0   
4          14.4        2009  Europe  Gasoline    All-wheel drive        2.0   

   fuel_efficiency_mpg  
0            13.231729  
1            13.688217  
2            14.246341  
3         

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   num_cylinders        9222 non-null   float64
 2   horsepower           8996 non-null   float64
 3   vehicle_weight       9704 non-null   float64
 4   acceleration         8774 non-null   float64
 5   model_year           9704 non-null   int64  
 6   origin               9704 non-null   object 
 7   fuel_type            9704 non-null   object 
 8   drivetrain           9704 non-null   object 
 9   num_doors            9202 non-null   float64
 10  fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 834.1+ KB


## Preparing the dataset

Preparation:

   * Fill missing values with zeros.
   * Do train/validation/test split with 60%/20%/20% distribution.
   * Use the train_test_split function and set the random_state parameter to 1.
   * Use DictVectorizer(sparse=True) to turn the dataframes into matrices.


In [3]:
df.isna().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [4]:
cols = ["num_cylinders", "horsepower","acceleration", "num_doors"]

for col in cols:
    df[col] = df[col].fillna(0)

from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2
len(df_train), len(df_val), len(df_test)

(5822, 1941, 1941)

In [5]:
from sklearn.feature_extraction import DictVectorizer

# Create the DictVectorizer
dv = DictVectorizer(sparse=True)

# Convert dataframes to dictionaries
train_dicts = df_train.drop("fuel_efficiency_mpg", axis=1).to_dict(orient='records')
val_dicts = df_val.drop("fuel_efficiency_mpg", axis=1).to_dict(orient='records')
test_dicts = df_test.drop("fuel_efficiency_mpg", axis=1).to_dict(orient='records')

# Fit and transform the training data, transform validation and test data
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

# Extract target values
y_train = df_train["fuel_efficiency_mpg"].values
y_val = df_val["fuel_efficiency_mpg"].values
y_test = df_test["fuel_efficiency_mpg"].values

## Question 1

Which feature is used for splitting the data?

   * 'vehicle_weight'
   * 'model_year'
   * 'origin'
   * 'fuel_type'


In [6]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train, y_train)

,criterion,'squared_error'
,splitter,'best'
,max_depth,1
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,1
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [7]:
feature_names = dv.get_feature_names_out()
split_feature = feature_names[dt.tree_.feature[0]]
print(f"Feature used for splitting the data: {split_feature}")

Feature used for splitting the data: vehicle_weight


## Question 2

Train a random forest regressor with these parameters:

  *  n_estimators=10
  *  random_state=1
  *  n_jobs=-1 (optional - to make training faster)

What's the RMSE of this model on the validation data?

  *  0.045
  *  0.45
  *  4.5
  *  45.0


In [8]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)


print(f"MSE on validation set: {mse}")
print(f"RMSE on validation set: {rmse}")

MSE on validation set: 0.21121168284297887
RMSE on validation set: 0.4595777223092726


## Question 3

Now let's experiment with the n_estimators parameter

  * Try different values of this parameter from 10 to 200 with step 10.
  * Set random_state to 1.
  * Evaluate the model on the validation dataset.

After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for calculating the answer.

  *  10
  *  25
  *  80
  *  200


In [9]:
rmse_scores = {}

for i in range(10,200,10):
    rd = RandomForestRegressor(n_estimators=i, random_state=1)
    rd.fit(X_train, y_train)
    y_pred = rd.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    rmse_scores[i] = rmse

    print(f"n_estimators: {i} => RMSE: {rmse:.3f}")

# Find when RMSE stops improving
print("\n--- Analysis ---")
prev_rmse = float('inf')
stop_improving_at = None

for n in sorted(rmse_scores.keys()):
    current_rmse = rmse_scores[n]
    if current_rmse >= prev_rmse:
        stop_improving_at = n
        print(f"\nRMSE stopped improving at n_estimators={n}")
        print(f"Previous RMSE (n={n-10}): {prev_rmse:.3f}")
        print(f"Current RMSE (n={n}): {current_rmse:.3f}")
        break
    prev_rmse = current_rmse

if stop_improving_at is None:
    print("\nRMSE kept improving throughout all values tested") # 80

n_estimators: 10 => RMSE: 0.460
n_estimators: 20 => RMSE: 0.454
n_estimators: 30 => RMSE: 0.452
n_estimators: 40 => RMSE: 0.449
n_estimators: 50 => RMSE: 0.447
n_estimators: 60 => RMSE: 0.445
n_estimators: 70 => RMSE: 0.445
n_estimators: 80 => RMSE: 0.445
n_estimators: 90 => RMSE: 0.445
n_estimators: 100 => RMSE: 0.445
n_estimators: 110 => RMSE: 0.444
n_estimators: 120 => RMSE: 0.444
n_estimators: 130 => RMSE: 0.444
n_estimators: 140 => RMSE: 0.443
n_estimators: 150 => RMSE: 0.443
n_estimators: 160 => RMSE: 0.443
n_estimators: 170 => RMSE: 0.443
n_estimators: 180 => RMSE: 0.442
n_estimators: 190 => RMSE: 0.442

--- Analysis ---

RMSE stopped improving at n_estimators=120
Previous RMSE (n=110): 0.444
Current RMSE (n=120): 0.444


## Question 4

Let's select the best max_depth:

   * Try different values of max_depth: [10, 15, 20, 25]
   * For each of these values,
        * try different values of n_estimators from 10 till 200 (with step 10)
        * calculate the mean RMSE
   * Fix the random seed: random_state=1

What's the best max_depth, using the mean RMSE?

  *  10
  *  15
  *  20
  *  25


In [10]:
max_depth_values = [10,15,20,25]
n_estimators = range(10,210,10)

results = {}

for max_depth in max_depth_values:
    rmse_scores = []
    for n in n_estimators:
        rf = RandomForestRegressor(n_estimators=n, max_depth=max_depth, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        rmse_scores.append(rmse)
    
    mean_rmse = np.mean(rmse_scores)
    results[max_depth] = mean_rmse
    print(f"Max Depth: {max_depth} => Mean RMSE: {mean_rmse:.3f}")


print("\n" + "="*40)
print("SUMMARY - Mean RMSE by max_depth:")
print("="*50)
for depth in max_depth_values:
    print(f"max_depth={depth}: Mean RMSE={results[depth]:.3f}")

best_max_depth = min(results, key=results.get)
print(f"\nBest max_depth: {best_max_depth} (Mean RMSE={results[best_max_depth]:.3f})")

Max Depth: 10 => Mean RMSE: 0.442
Max Depth: 15 => Mean RMSE: 0.445
Max Depth: 20 => Mean RMSE: 0.446
Max Depth: 25 => Mean RMSE: 0.446

SUMMARY - Mean RMSE by max_depth:
max_depth=10: Mean RMSE=0.442
max_depth=15: Mean RMSE=0.445
max_depth=20: Mean RMSE=0.446
max_depth=25: Mean RMSE=0.446

Best max_depth: 10 (Mean RMSE=0.442)


## Question 5

For this homework question, we'll find the most important feature:

   * Train the model with these parameters:
       *  n_estimators=10,
       *  max_depth=20,
       *  random_state=1,
       *  n_jobs=-1 (optional)
   *  Get the feature importance information from this model

What's the most important feature (among these 4)?

   *  vehicle_weight
   *  horsepower
   *  acceleration
   *  engine_displacement


In [12]:
rf = RandomForestRegressor(
    n_estimators=10,
    max_depth=20,
    random_state=1,
    n_jobs=-1)
rf.fit(X_train, y_train)

# Get feature importances
feature_importances = rf.feature_importances_

# Get feature names from DictVectorizer
feature_names = dv.get_feature_names_out()

# Create a dictionary to see importances with their feature names
importance_dict = dict(zip(feature_names, feature_importances))

# Find the most important among the 4 specified features
features_to_check = ['vehicle_weight', 'horsepower', 'acceleration', 'engine_displacement']

for feature in features_to_check:
    if feature in importance_dict:
        print(f"{feature}: {importance_dict[feature]:.4f}")

# Or find the maximum
max_importance = max([(f, importance_dict[f]) for f in features_to_check if f in importance_dict], 
                     key=lambda x: x[1])
print(f"\nAnswer 5: {max_importance[0]} with importance {max_importance[1]:.4f}")

vehicle_weight: 0.9591
horsepower: 0.0160
acceleration: 0.0115
engine_displacement: 0.0033

Answer 5: vehicle_weight with importance 0.9591


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

   * Install XGBoost
   * Create DMatrix for train and validation
   * Create a watchlist
   * Train a model with these parameters for 100 rounds:

```python
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

```
Now change eta from 0.3 to 0.1.

Which eta leads to the best RMSE score on the validation dataset?

   * 0.3
   * 0.1
   * Both give equal value


In [16]:
import xgboost as xgb

features = dv.get_feature_names_out()
features = features.tolist()

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [17]:
# Create a watchlist 
watchlist = [(dtrain, "train") , (dval, "val")]

In [18]:
# Train a model with these parameters for 100 rounds
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100, verbose_eval=5, evals=watchlist)

[0]	train-rmse:1.81393	val-rmse:1.85444
[5]	train-rmse:0.51381	val-rmse:0.55664
[10]	train-rmse:0.37115	val-rmse:0.43896
[15]	train-rmse:0.34666	val-rmse:0.43362
[20]	train-rmse:0.33553	val-rmse:0.43376
[25]	train-rmse:0.32268	val-rmse:0.43683
[30]	train-rmse:0.31475	val-rmse:0.43752
[35]	train-rmse:0.30960	val-rmse:0.43784
[40]	train-rmse:0.30202	val-rmse:0.43968
[45]	train-rmse:0.29126	val-rmse:0.44024
[50]	train-rmse:0.28456	val-rmse:0.44140
[55]	train-rmse:0.27618	val-rmse:0.44225
[60]	train-rmse:0.26768	val-rmse:0.44290
[65]	train-rmse:0.26174	val-rmse:0.44352
[70]	train-rmse:0.25489	val-rmse:0.44531
[75]	train-rmse:0.24792	val-rmse:0.44628
[80]	train-rmse:0.24254	val-rmse:0.44689
[85]	train-rmse:0.23644	val-rmse:0.44749
[90]	train-rmse:0.23193	val-rmse:0.44839
[95]	train-rmse:0.22475	val-rmse:0.44904
[99]	train-rmse:0.21950	val-rmse:0.45018


In [19]:
# RMSE for ets : 0.3 
y_pred = model.predict(dval)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE for ets = 0.3: {rmse}")

RMSE for ets = 0.3: 0.45017755678087246


In [20]:
# Train the model with changing eta from 0.3 to 0.1
xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100, verbose_eval=5, evals=watchlist)
y_pred = model.predict(dval)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE for ets = 0.1: {rmse}")

[0]	train-rmse:2.28944	val-rmse:2.34561
[5]	train-rmse:1.41247	val-rmse:1.44988
[10]	train-rmse:0.91008	val-rmse:0.94062
[15]	train-rmse:0.63402	val-rmse:0.66672
[20]	train-rmse:0.48983	val-rmse:0.53064
[25]	train-rmse:0.41881	val-rmse:0.46891
[30]	train-rmse:0.38342	val-rmse:0.44289
[35]	train-rmse:0.36435	val-rmse:0.43250
[40]	train-rmse:0.35343	val-rmse:0.42746
[45]	train-rmse:0.34621	val-rmse:0.42595
[50]	train-rmse:0.33998	val-rmse:0.42498
[55]	train-rmse:0.33480	val-rmse:0.42449
[60]	train-rmse:0.33054	val-rmse:0.42456
[65]	train-rmse:0.32602	val-rmse:0.42493
[70]	train-rmse:0.32202	val-rmse:0.42503
[75]	train-rmse:0.31895	val-rmse:0.42526
[80]	train-rmse:0.31667	val-rmse:0.42563
[85]	train-rmse:0.31440	val-rmse:0.42574
[90]	train-rmse:0.31059	val-rmse:0.42586
[95]	train-rmse:0.30625	val-rmse:0.42611
[99]	train-rmse:0.30419	val-rmse:0.42623
RMSE for ets = 0.1: 0.42622800553359225
